In [1]:
# Import necessary libraries
import cv2
from deepface import DeepFace

# Load face cascade classifier
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Initialize variables
frame_count = 0
frame_accumulator = []
emotion_labels = []

# Open webcam for video capture
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError("Cannot Open Webcam")

while True:
    ret, frame = cap.read()  # Read frame from the webcam
    
    # Perform face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray, 1.1, 4)
    
    # Iterate over detected faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        # Perform emotion analysis on the face region
        face_img = frame[y:y+h, x:x+w]
        emotion_result = DeepFace.analyze(face_img, actions=['emotion'],enforce_detection=False)
        emotion_label = emotion_result[0]['dominant_emotion']
        
        # Accumulate emotions over multiple frames
        frame_accumulator.append(face_img)
        emotion_labels.append(emotion_label)
        
    # Increment frame count
    frame_count += 1
    
    # Analyze emotions after accumulating a certain number of frames
    if frame_count == 10:
        # Calculate the dominant emotion from accumulated frames
        dominant_emotion = max(set(emotion_labels), key=emotion_labels.count)
        
        # Reset frame count and accumulators
        frame_count = 0
        frame_accumulator = []
        emotion_labels = []
        
        # Draw emotion label on the frame
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(frame, dominant_emotion, (50, 50), font, 3, (0, 0, 255), 2, cv2.LINE_4)
    
    # Display the frame
    cv2.imshow('Original Video', frame)
    
    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and destroy windows
cap.release()
cv2.destroyAllWindows()


Action: emotion: 100%|███████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 17.29it/s]


ValueError: max() arg is an empty sequence